In [1]:
import pandas as pd
import os
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
# Arbeitsverzeichnis ändern 
#neues_verzeichnis = "c:\\Users\\lpres\\OneDrive - JGU\\WMDV\\WebMining"
neues_verzeichnis = "C:\\Users\\Lucian\\Desktop\\WebMining"
os.chdir(neues_verzeichnis)

# Überprüfen, ob das Verzeichnis geändert wurde
print("Neues Arbeitsverzeichnis:", os.getcwd())

Neues Arbeitsverzeichnis: C:\Users\Lucian\Desktop\WebMining


In [3]:
# Alle vorhandenen owned_games_x Dateien finden
owned_files = [f for f in os.listdir('.') if re.match(r'owned_games_\d+\.csv', f)]
owned_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

print(f"Vorhandene Dateien: {owned_files}")

if not owned_files:
    print("Keine Dateien im Format 'owned_games_x.csv' gefunden. Das Skript wird beendet.")

# Alle Dateien einlesen und in einem DataFrame speichern
pre_df = pd.DataFrame()  # Leerer DataFrame

for file in owned_files:
    file_df = pd.read_csv(file, dtype={
         "steam_64_id": str,
         "appid": "Int64",  
         "playtime_forever": "Int64",
         "playtime_2weeks": "Int64"
    })
    pre_df = pd.concat([pre_df, file_df], ignore_index=True)

print("alle Dateien eingelesen")
print(pre_df.shape)

Vorhandene Dateien: ['owned_games_1.csv', 'owned_games_2.csv', 'owned_games_3.csv', 'owned_games_4.csv', 'owned_games_5.csv', 'owned_games_6.csv', 'owned_games_7.csv', 'owned_games_8.csv']


C:\Users\Lucian\AppData\Local\Temp\ipykernel_7676\35310863.py:14: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file, dtype={


alle Dateien eingelesen
(8980716, 6)


In [20]:
# Duplikate und "No Games" rausschmeißen
h_df = pre_df.drop_duplicates(keep="first")
h_df = h_df[h_df["name"] != "No Games"]
dropped_rows = pre_df.shape[0] - h_df.shape[0]
print(h_df.shape)
print(f"{dropped_rows} Duplikate wurden entfernt")

(5528936, 6)
3451780 Duplikate wurden entfernt


In [21]:
# Sortieren nach höchster Spielzeit und dann Duplikate entfernen. Falls steam_id mehrfach abgefragt wurde, sollen die einträge behalten werden, die am neuesten sind (max(playtime_forever))
df = h_df.sort_values(by="playtime_forever", ascending=False).drop_duplicates(subset=["steam_64_id", "name"], keep="first")

# Anzahl der entfernten Duplikate berechnen
dropped_rows = h_df.shape[0] - df.shape[0]

print(df.shape)
print(f"{dropped_rows} Duplikate wurden entfernt (behalten wurde die Zeile mit der höchsten Spielzeit pro Spieler-Spiel-Kombination).")


(5469541, 6)
59395 Duplikate wurden entfernt (behalten wurde die Zeile mit der höchsten Spielzeit pro Spieler-Spiel-Kombination).


In [ ]:
# Datensatz reduzieren auf Spiele, die von mindestens 2500 Spielern gespielt werden
spiel_counts = df['name'].value_counts()
filtered_games = df[df['name'].isin(spiel_counts[spiel_counts >= 2500].index)]

dropped_rows = df.shape[0] - filtered_games.shape[0]
print(f"{dropped_rows} weitere Zeilen wurden entfernt")

4050525 weitere Zeilen wurden entfernt


In [6]:
# Erstellen einer Liste, die jedes relevante Spiel genau einmal enthält 
unique_games = filtered_games.drop_duplicates(subset=["name"], keep="first")
unique_games = unique_games[unique_games["name"] != "No Games"]
unique_games.shape

(275, 6)

In [24]:
unique_players_per_game = filtered_games.groupby('name')['steam_64_id'].nunique()
unique_players_per_game = unique_players_per_game.sort_values(ascending=False)
unique_players_per_game

name
Counter-Strike 2                             37329
Dota 2                                       35295
PUBG: BATTLEGROUNDS                          29224
Wallpaper Engine                             18496
Left 4 Dead 2                                18491
                                             ...  
Dark and Darker                               2563
World of Tanks                                2559
HELLDIVERS™ 2                                 2535
A Story About My Uncle                        2518
Grand Theft Auto IV: The Complete Edition     2516
Name: steam_64_id, Length: 266, dtype: int64

In [54]:
# Liste, die jedes relevante Spiel genau einmal enthält speichern
unique_games.to_csv('unique_games.csv', index=False)

In [25]:
# Wie viele Leute spielen welches Spiel wie oft im gegebenen Datensatz?
unique_spiel_counts = filtered_games["name"].value_counts()
unique_spiel_counts

name
Counter-Strike 2                             37329
Dota 2                                       35295
PUBG: BATTLEGROUNDS                          29224
Wallpaper Engine                             18496
Left 4 Dead 2                                18491
                                             ...  
Dark and Darker                               2563
World of Tanks                                2559
HELLDIVERS™ 2                                 2535
A Story About My Uncle                        2518
Grand Theft Auto IV: The Complete Edition     2516
Name: count, Length: 266, dtype: int64